In [80]:
pip install --upgrade spreg

Note: you may need to restart the kernel to use updated packages.


In [82]:
pip install numpy libpysal spreg

In [84]:
pip install esda

In [86]:
pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [88]:
import numpy as np
import pandas as pd
from spreg import OLS, ML_Lag, ML_Error
from libpysal.weights import lat2W
from scipy.stats import zscore
from esda.moran import Moran


# Generate sample data
np.random.seed(123)
n = 500
X = np.random.rand(n, 1)
y = 0.5 + 0.95 * X.flatten() + np.random.normal(0, 1, n)

# Create spatial weight matrix (e.g., Queen or Rook contiguity for lattice data)
# Assuming a regular grid for simplicity
w = lat2W(25, 20)  # 25x20 grid for 500 observations

# Normalize weights
w.transform = 'r'

# OLS Model
ols_model = OLS(
    y.reshape(-1, 1),
    np.hstack((np.ones((n, 1)), X)),
    name_y="Y",
    name_x=["const", "X"],
)
print("OLS Results:")
print(ols_model.summary)


# FAR Model (Spatial Lag)
lag_model = ML_Lag(
    y.reshape(-1, 1),
    np.hstack((np.ones((n, 1)), X)),
    w,
    name_y="Y",
    name_x=["const", "X"],
)
print("\nSpatial Lag (FAR) Results:")
print(lag_model.summary)

# SEM Model (Spatial Error)
sem_model = ML_Error(
    y.reshape(-1, 1),
    np.hstack((np.ones((n, 1)), X)),
    w,
    name_y="Y",
    name_x=["const", "X"],
)
print("\nSpatial Error (SEM) Results:")
print(sem_model.summary)

ols_residuals = y - ols_model.predy.flatten()


# Spatial Diagnostics
moran = Moran(ols_residuals, w)
print(f"\nMoran's I on OLS residuals: {moran.I:.4f}")
print(f"P-value: {moran.p_sim:.4f}")

OLS Results:
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :           Y                Number of Observations:         500
Mean dependent var  :      0.9678                Number of Variables   :           2
S.D. dependent var  :      1.0582                Degrees of Freedom    :         498
R-squared           :      0.0827
Adjusted R-squared  :      0.0809
Sum squared residual:     512.568                F-statistic           :     44.8997
Sigma-square        :       1.029                Prob(F-statistic)     :   5.625e-11
S.E. of regression  :       1.015                Log likelihood        :    -715.676
Sigma-square ML     :       1.025                Akaike info criterion :    1435.351
S.E of regression ML:      1.0125                Schwarz criterion     :    1443.781

-----------------------------------------------

C:\Users\zauka\anaconda3\Lib\site-packages\scipy\optimize\_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "



Spatial Error (SEM) Results:
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR (METHOD = full)
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :           Y                Number of Observations:         500
Mean dependent var  :      0.9678                Number of Variables   :           2
S.D. dependent var  :      1.0582                Degrees of Freedom    :         498
Pseudo R-squared    :      0.0827
Log likelihood      :   -715.3145
Sigma-square ML     :      1.0229                Akaike info criterion :    1434.629
S.E of regression   :      1.0114                Schwarz criterion     :    1443.058

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
          